In [17]:
import torch  # 'torch' 라이브러리 import 추가
from torch import nn, optim
from torch.utils.data import Dataset, random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from pathlib import Path
import sys
import os

# 하단의 경우엔 .py파일에서 진행
# BASE_PATH 설정 (필요에 따라 수정)
#BASE_PATH = str(Path(__file__).resolve().parent.parent.parent)
#print(BASE_PATH, "!!!!!")
#sys.path.append(BASE_PATH)

# 하단의 경우엔 .ipynb 파일에서 진행
# 상단 코드의 경우엔 jupyter에서 인식이 안됨
# Jupyter Notebook에서 현재 작업 디렉토리 설정
BASE_PATH = str(Path(os.getcwd()).resolve().parent.parent)  # 현재 경로의 상위 두 단계를 기본 경로로 설정
print(BASE_PATH, "!!!!!")
sys.path.append(BASE_PATH)


C:\Users\didsu\code_projects\uni_24\DL\link_dl !!!!!


Target	Feature
     • Survived:			사망,			생존– Feature    
 • PassengerId: 승객 

Feature번    호
• Name:     이름
• Pclass:	티켓
        스
Ø 1	=	1등석,	2	=	2등석,	3	=    	3등석
• S
        	성별
Ø male	=	남성,	femal    e	=	여성
•     Age:	나이
• SibSp:	동승한	자매	    /	배우자의	수
• Parch:	동승한	부    모	/	자식의	수
• Tic    ket:	티켓	번호
•     Fare:	승객	요금
•     Cabin:	방	호수
•
        ked:	탑승지
4
 Ø C	=	셰르부르,	Q	=	퀸즈타운,	S	=	사우스햄프턴

# 데이터셋 클래스 정의

In [12]:
# 데이터셋 클래스 정의
class TitanicDataset(Dataset):
    def __init__(self, csv_file, is_test=False):
        self.data = pd.read_csv(csv_file)

        if is_test:
            self.passenger_ids = self.data['PassengerId']
            # 테스트 데이터셋에서는 'Survived' 열이 없으므로 제거하지 않습니다.
            columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked']
            self.data = self.data.drop(columns=columns_to_drop, axis=1, errors='ignore')
        else:
            # 훈련 데이터셋에서는 'Survived' 열이 필요하므로 제거하지 않습니다.
            columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked']
            self.data = self.data.drop(columns=columns_to_drop, axis=1, errors='ignore')

        # 'Sex' 인코딩: male=0, female=1
        self.data['Sex'] = self.data['Sex'].map({'male': 0, 'female': 1})

        # 'Age' 결측값을 평균으로 채움
        self.data['Age'] = self.data['Age'].fillna(self.data['Age'].mean())

        if not is_test:
            # 입력 특징과 타겟 분리
            self.X = self.data[['Pclass', 'Sex', 'Age']].values.astype(float)
            self.y = self.data['Survived'].values.astype(float)
        else:
            # 테스트 데이터셋에서는 타겟이 없음
            self.X = self.data[['Pclass', 'Sex', 'Age']].values.astype(float)
            self.y = None

        # 정규화
        self.scaler = StandardScaler()
        self.X = self.scaler.fit_transform(self.X)

        # 텐서로 변환
        self.X = torch.tensor(self.X, dtype=torch.float32)
        if not is_test:
            self.y = torch.tensor(self.y, dtype=torch.float32).unsqueeze(1)  # (N, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            # 테스트 데이터셋에서는 PassengerId와 함께 반환
            return self.X[idx], self.passenger_ids.iloc[idx]

def get_data(csv_path):
    dataset = TitanicDataset(csv_file=csv_path, is_test=False)
    print(dataset)

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, validation_dataset = random_split(dataset, [train_size, val_size])
    print(f"Train size: {len(train_dataset)}, Validation size: {len(validation_dataset)}")

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

    return train_data_loader, validation_data_loader

def get_test_data(csv_path):
    test_dataset = TitanicDataset(csv_file=csv_path, is_test=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))
    return test_data_loader

In [13]:
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.ELU(),  # ReLU에서 ELU로 변경
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.ELU(),  # ReLU에서 ELU로 변경
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
            nn.Sigmoid()  # 이진 분류를 위해 Sigmoid 활성화 추가
        )

    def forward(self, x):
        return self.model(x)

In [14]:
def get_model_and_optimizer():
    my_model = MyModel(n_input=3, n_output=1)  # 입력 특징 수에 맞게 조정
    optimizer = optim.Adam(my_model.parameters(), lr=wandb.config.learning_rate)  # Adam 옵티마이저 사용

    return my_model, optimizer

def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.BCELoss()  # 이진 분류를 위한 손실 함수
    next_print_epoch = 100

    for epoch in range(1, n_epochs + 1):
        model.train()
        loss_train = 0.0
        all_preds_train = []
        all_targets_train = []

        for train_batch in train_data_loader:
            input, target = train_batch
            output_train = model(input)
            loss = loss_fn(output_train, target)
            loss_train += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            preds = (output_train > 0.5).float()
            all_preds_train.extend(preds.cpu().numpy())
            all_targets_train.extend(target.cpu().numpy())

        train_loss = loss_train / len(train_data_loader)
        train_accuracy = accuracy_score(all_targets_train, all_preds_train)
        train_precision = precision_score(all_targets_train, all_preds_train)
        train_recall = recall_score(all_targets_train, all_preds_train)
        train_f1 = f1_score(all_targets_train, all_preds_train)

        model.eval()
        loss_validation = 0.0
        all_preds_val = []
        all_targets_val = []

        with torch.no_grad():
            for validation_batch in validation_data_loader:
                input, target = validation_batch
                output_validation = model(input)
                loss = loss_fn(output_validation, target)
                loss_validation += loss.item()

                preds = (output_validation > 0.5).float()
                all_preds_val.extend(preds.cpu().numpy())
                all_targets_val.extend(target.cpu().numpy())

        val_loss = loss_validation / len(validation_data_loader)
        val_accuracy = accuracy_score(all_targets_val, all_preds_val)
        val_precision = precision_score(all_targets_val, all_preds_val)
        val_recall = recall_score(all_targets_val, all_preds_val)
        val_f1 = f1_score(all_targets_val, all_preds_val)

        wandb.log({
            "Epoch": epoch,
            "Training Loss": train_loss,
            "Validation Loss": val_loss,
            "Training Accuracy": train_accuracy,
            "Validation Accuracy": val_accuracy,
            "Training Precision": train_precision,
            "Validation Precision": val_precision,
            "Training Recall": train_recall,
            "Validation Recall": val_recall,
            "Training F1 Score": train_f1,
            "Validation F1 Score": val_f1
        })

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training Loss: {train_loss:.4f}, "
                f"Validation Loss: {val_loss:.4f}, "
                f"Training Acc: {train_accuracy:.4f}, "
                f"Validation Acc: {val_accuracy:.4f}, "
                f"Training Precision: {train_precision:.4f}, "
                f"Validation Precision: {val_precision:.4f}, "
                f"Training Recall: {train_recall:.4f}, "
                f"Validation Recall: {val_recall:.4f}, "
                f"Training F1: {train_f1:.4f}, "
                f"Validation F1: {val_f1:.4f}"
            )
            next_print_epoch += 100

def predict_test(model, test_data_loader, output_path):
    model.eval()
    all_preds = []
    all_passenger_ids = []

    with torch.no_grad():
        for test_batch in test_data_loader:
            input, passenger_ids = test_batch
            output = model(input)
            preds = (output > 0.5).int()
            # preds를 1차원으로 변환하여 리스트에 추가
            all_preds.extend(preds.cpu().numpy().flatten())
            all_passenger_ids.extend(passenger_ids.cpu().numpy())

    # 결과를 CSV 파일로 저장
    preds_df = pd.DataFrame({
        'PassengerId': all_passenger_ids,
        'Survived': all_preds
    })
    preds_df.to_csv(output_path, index=False)
    print(f"Test predictions saved to {output_path}")


def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': args.learning_rate,
        'n_hidden_unit_list': args.hidden_units,  # 리스트로 받도록 수정
    }

    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="titanic_model_training",
        notes="Titanic survival prediction experiment",
        tags=["titanic", "binary_classification"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    # 훈련 및 검증 데이터 로더
    train_data_loader, validation_data_loader = get_data(csv_path=args.train_csv_path)

    # 모델 및 옵티마이저 초기화
    model, optimizer = get_model_and_optimizer()

    # 훈련 루프 실행
    training_loop(
        model=model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader
    )

    # 테스트 데이터 로더 (타겟 없음)
    test_data_loader = get_test_data(csv_path=args.test_csv_path)

    # 테스트 데이터에 대한 예측 수행 및 저장
    predict_test(model, test_data_loader, output_path=args.output_path)

    wandb.finish()


In [18]:
if __name__ == "__main__":
    # 명령어 인자를 직접 설정하는 방식으로 수정
    args = argparse.Namespace(
        train_csv_path="train.csv",  # 실제 경로로 수정
        test_csv_path="test.csv",    # 실제 경로로 수정
        output_path="test_predictions_ELU_test_using_args.csv",  # 실제 경로로 수정
        wandb=True,                  # WandB 사용 여부
        batch_size=64,               # 배치 사이즈
        epochs=200,                  # 학습 에폭
        learning_rate=0.001,         # 학습률
        hidden_units=[128, 64]       # 은닉층 크기
    )

    # main 함수 호출
    main(args)


Namespace(train_csv_path='train.csv', test_csv_path='test.csv', output_path='test_predictions_ELU_test_using_args.csv', wandb=True, batch_size=64, epochs=200, learning_rate=0.001, hidden_units=[128, 64])
{'epochs': 200, 'batch_size': 64, 'learning_rate': 0.001, 'n_hidden_unit_list': [128, 64]}
Train size: 712, Validation size: 179
Epoch 100, Training Loss: 0.4818, Validation Loss: 0.4000, Training Acc: 0.7907, Validation Acc: 0.7989, Training Precision: 0.7807, Validation Precision: 0.7636, Training Recall: 0.6426, Validation Recall: 0.6462, Training F1: 0.7050, Validation F1: 0.7000
Epoch 200, Training Loss: 0.4354, Validation Loss: 0.3927, Training Acc: 0.7978, Validation Acc: 0.7989, Training Precision: 0.8844, Validation Precision: 0.7843, Training Recall: 0.5523, Validation Recall: 0.6154, Training F1: 0.6800, Validation F1: 0.6897
Test predictions saved to test_predictions_ELU_test_using_args.csv


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇██
Training Accuracy,▃▆▅▁▃▄▄▄▃▄▅▅▁▄▄▄▆▆▄▆▅▅▇▅▅▃▇▆▃▃▂▇▆▆▄▇▆▄▅█
Training F1 Score,▃▆▅▆▇▁▄▃▁▃▅▁▄▅▄▅▆▇▂▄▇▆▇▇▄▃▆▃▄▇█▇▅▆▆▅▆▂▅▇
Training Loss,▄▅▄▇▅▃▅▇▇▆▂▃█▇▅▅▃▁▅▄▄▅▅▃▁▃▁▃▆▃▂▁▃█▄▅▅▃▄▄
Training Precision,▁▁▄▂▁▅▅▂▅▄▅▅▃▄▅▂▅▄▇▄▂▄▆▄▄▂█▄▅▄▆▂▅▇▅▃▄▂▂▄
Training Recall,███▅▇▄▄▅▂▃▁█▆▆▃▇█▅▃▂▄▇▃▄▂▇▄▄▂▃▄▄▅▂▆▄▅▆▆▇
Validation Accuracy,▆▆▆▁▇▄▆▆▅▆▄▃▃▆▆▅▃▃▆▃▅▄▅▃▄▅▄▅▄▃█▄▄▅▆▆▃▅▇▃
Validation F1 Score,▁█▅▇▅▃▄▃▇▄▄▄▄▄▃█▄▃▄▄█▂▄▅▄▄▅▄▄▇▄▄▄▄▄▃▄▃▃▄
Validation Loss,█▁▁▁▂▂▂▂▄▃▂▃▃▁▃▁▂▃▃▃▂▂▄▃▁▁▃▃▁▂▁▂▂▂▁▄▂▂▂▄
Validation Precision,▁▃▄▅▂▄▅▆█▆▇▆▄█▃▅▃▅██▃▆▅▅▆▆▄▅▇▅▄▇▄▄▃▃▅▄▄▃
Validation Recall,▇█▇▃▄▇▃█▄▄▄▄▃▃▄▃█▇▃▃▁▃▄█▇▄▄▇▃▄▃▃▄▄▃▅▃███


![competition78](./78Scores_leaderboard.png)




절대 경로로 진행하려고 했지만 너무 안되었고 상대경로는 되지만 절대경로는 안되는것을 미루어보아
jupyter라는 가상 서버를 이용하기 때문에 경로를 잘못인식을 하는건가 싶다.
왜냐하면 getcwd()를 했음에도 문제가 없었기 때문이다.

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="Enable or disable WandB logging")
    parser.add_argument(
        "-b", "--batch_size", type=int, default=32, help="Batch size (int, default: 32)")
    parser.add_argument(
        "-e", "--epochs", type=int, default=100, help="Number of training epochs (int, default: 100)")
    parser.add_argument(
        "--learning_rate", type=float, default=1e-3, help="Learning rate (float, default: 1e-3)")
    parser.add_argument(
        "--hidden_units", type=int, nargs='+', default=[64, 32], help="List of hidden units (default: [64, 32])")
    # 수정된 인자
    parser.add_argument(
        "--train_csv_path", type=str, required=True, help="Path to the training CSV dataset")
    parser.add_argument(
        "--test_csv_path", type=str, required=True, help="Path to the testing CSV dataset")
    parser.add_argument(
        "--output_path", type=str, required=True, help="Path to save test predictions")
    args = parser.parse_args()
    main(args)

위 코드로 진행하려고 한다면 terminal에서 python f_my_model_training_with_argparse_wandb.py --train_csv_path train.csv --test_csv_path test.csv --output_path test_predictions_ELU.csv --wandb --batch_size 64 --epochs 200 --learning_rate 0.001 --hidden_units 128 64 이렇게 작성하면 된다.

In [ ]:
아래처럼 나와야한다

# 1

In [76]:
# Section 1: Imports and Dataset Class Definitions
import os
import torch
import pandas as pd
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from datetime import datetime
import wandb
import argparse
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 2

In [77]:
# TitanicDataset Class for Train and Validation Data
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return {'input': self.X[idx], 'target': self.y[idx]}
    
    def __str__(self):
        return f"Data Size: {len(self.X)}, Input Shape: {self.X.shape}, Target Shape: {self.y.shape}"

In [53]:
# TitanicTestDataset Class for Test Data
class TitanicTestDataset(Dataset):
    def __init__(self, X):
        self.X = torch.FloatTensor(X)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return {'input': self.X[idx]}
    
    def __str__(self):
        return f"Data Size: {len(self.X)}, Input Shape: {self.X.shape}"

# 3

In [78]:
# version 2
class TitanicTestDataset(Dataset):
    def __init__(self, X, passenger_ids):
        self.X = torch.FloatTensor(X)
        self.passenger_ids = passenger_ids  # PassengerId 추가
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        feature = self.X[idx]
        passenger_id = self.passenger_ids[idx]  # PassengerId 반환
        return {'input': feature, 'PassengerId': passenger_id}  # PassengerId와 입력 특징 반환


# 4

In [94]:
def get_preprocessed_dataset_1(df):
    # 성별 인코딩 (male=0, female=1)
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    return df

def get_preprocessed_dataset_2(df):
    # 결측값 처리 (Age의 결측값을 평균으로 채움)
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    return df

def get_preprocessed_dataset_3(df):
    # 결측값 처리 (Fare의 결측값을 평균으로 채움)
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    return df

def get_preprocessed_dataset_4(df):
    # 불필요한 열 제거, 단 PassengerId는 제거하지 않음
    columns_to_drop = ['Name', 'Ticket', 'Cabin', 'Embarked']  # 'PassengerId' 제외
    df = df.drop(columns=columns_to_drop, axis=1, errors='ignore')
    return df

def get_preprocessed_dataset_5(df):
    # Pclass를 범주형에서 연속형으로 처리
    df['Pclass'] = df['Pclass'].astype(float)
    return df

def get_preprocessed_dataset_6(df):
    # 정규화 (필요한 경우)
    scaler = StandardScaler()
    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
    df[features] = scaler.fit_transform(df[features])
    return df


# Section 2: Data Preprocessing

In [95]:
# Section 2: Data Preprocessing
def get_preprocessed_dataset():
    CURRENT_FILE_PATH = os.getcwd()
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
    
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    all_df = pd.concat([train_df, test_df], sort=False)
    
    # Call pre-processing functions (define as needed)
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)
    
    return all_df

In [96]:
# Section 3: Model Definition
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_input, 30),
            nn.ELU(),            # ReLU 대신 ELU 사용
            nn.Linear(30, 30),
            nn.ELU(),            # ReLU 대신 ELU 사용
            nn.Linear(30, n_output),
            nn.Softmax(dim=1)    # Softmax는 그대로 유지
        )

    def forward(self, x):
        return self.model(x)

# Section 3: Model Definition

In [107]:
# Section 3: Model Definition version 2
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_input, 256),
            nn.ELU(),
            nn.Dropout(0.3),  # Dropout 추가
            nn.Linear(256, 64),
            nn.ELU(),
            nn.Dropout(0.3),  # Dropout 추가
            nn.Linear(64, n_output),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.model(x)

In [97]:
# Section 4: Training and Evaluation Loops
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(1, n_epochs + 1):
        model.train()
        for batch in train_data_loader:
            inputs, targets = batch['input'], batch['target']
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch} - Training loss: {loss.item()}")

    print("Training complete.")

# Section 4: Training and Evaluation Loops

In [108]:
# Section 4: Training and Evaluation Loops_v2_wandB에 추가하기
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(1, n_epochs + 1):
        model.train()
        train_loss = 0.0
        for batch in train_data_loader:
            inputs, targets = batch['input'], batch['target']
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            train_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_data_loader)

        # Validation Loss 계산
        model.eval()
        validation_loss = 0.0
        with torch.no_grad():
            for batch in validation_data_loader:
                inputs, targets = batch['input'], batch['target']
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                validation_loss += loss.item()
        
        validation_loss /= len(validation_data_loader)

        # WandB에 기록
        wandb.log({"Training Loss": train_loss, "Validation Loss": validation_loss, "Epoch": epoch})

        print(f"Epoch {epoch} - Training Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}")


In [58]:
# Section 5: Model Testing and Prediction - version 1
def predict_test(model, test_data_loader,output_path="test_predictions_ELU_testing_almost_final.csv"):
    model.eval()
    predictions = []
    preds_df.to_csv(output_path, index=False)
    print(f"Test predictions saved to {os.path.abspath(output_path)}")  # 파일의 절대 경로 출력
    with torch.no_grad():
        for batch in test_data_loader:
            inputs = batch['input']
            outputs = model(inputs)
            predicted_labels = torch.argmax(outputs, dim=1)
            predictions.extend(predicted_labels.cpu().numpy())
    return predictions

In [61]:
import pandas as pd

# Section 5: Model Testing and Prediction _version 2
def predict_test(model, test_data_loader, output_path="test_predictions_ELU_testing_almost_final.csv"):
    model.eval()
    predictions = []
    
    # 예측 수행
    with torch.no_grad():
        for batch in test_data_loader:
            inputs = batch['input']
            outputs = model(inputs)
            predicted_labels = torch.argmax(outputs, dim=1)
            predictions.extend(predicted_labels.cpu().numpy())
    
    # 예측 결과를 DataFrame으로 변환
    preds_df = pd.DataFrame({"Predicted": predictions})
    
    # DataFrame을 CSV 파일로 저장
    preds_df.to_csv(output_path, index=False)
    print(f"Test predictions saved to {os.path.abspath(output_path)}")  # 파일의 절대 경로 출력

    return predictions


In [72]:
import pandas as pd

# Model Testing and Prediction 3차 version
def predict_test(model, test_data_loader, output_path="test_predictions_ELU_testing_almost_final.csv"):
    model.eval()
    predictions = []
    passenger_ids = []  # PassengerId 저장을 위한 리스트

    # 예측 수행
    with torch.no_grad():
        for batch in test_data_loader:
            inputs = batch['input']
            passenger_ids_batch = batch['PassengerId']  # PassengerId 가져오기
            outputs = model(inputs)
            predicted_labels = torch.argmax(outputs, dim=1)
            
            # 예측 값과 PassengerId 저장
            predictions.extend(predicted_labels.cpu().numpy())
            passenger_ids.extend(passenger_ids_batch.cpu().numpy())
    
    # 예측 결과를 DataFrame으로 변환
    preds_df = pd.DataFrame({
        "PassengerId": passenger_ids,
        "Survived": predictions
    })
    
    # DataFrame을 CSV 파일로 저장
    preds_df.to_csv(output_path, index=False)
    print(f"Test predictions saved to {os.path.abspath(output_path)}")  # 파일의 절대 경로 출력

    return predictions



In [99]:
# version 4 
import pandas as pd

# Model Testing and Prediction
def predict_test(model, test_data_loader, output_path="test_predictions_ELU_testing_almost_final.csv"):
    model.eval()
    predictions = []
    passenger_ids = []  # PassengerId 저장을 위한 리스트

    # 예측 수행
    with torch.no_grad():
        for batch in test_data_loader:
            inputs = batch['input']
            passenger_ids_batch = batch['PassengerId']  # PassengerId 가져오기
            outputs = model(inputs)
            predicted_labels = torch.argmax(outputs, dim=1)
            
            # 예측 값과 PassengerId 저장
            predictions.extend(predicted_labels.cpu().numpy())
            passenger_ids.extend(passenger_ids_batch)

    # 예측 결과를 DataFrame으로 변환
    preds_df = pd.DataFrame({
        "PassengerId": passenger_ids,
        "Survived": predictions
    })
    
    # DataFrame을 CSV 파일로 저장
    preds_df.to_csv(output_path, index=False)
    print(f"Test predictions saved to {os.path.abspath(output_path)}")  # 파일의 절대 경로 출력

    return predictions


# Section 5: Model Testing and Predictio

In [109]:
# predict_test version 5
# Section 5: Model Testing and Predictio
import pandas as pd

# Model Testing and Prediction
def predict_test(model, test_data_loader, output_path="test_predictions_ELU_testing_almost_final.csv"):
    model.eval()
    predictions = []
    passenger_ids = []  # PassengerId 저장을 위한 리스트

    # 예측 수행
    with torch.no_grad():
        for batch in test_data_loader:
            inputs = batch['input']
            passenger_ids_batch = batch['PassengerId']  # PassengerId 가져오기
            outputs = model(inputs)
            predicted_labels = torch.argmax(outputs, dim=1)
            
            # 예측 값과 PassengerId 저장
            predictions.extend(predicted_labels.cpu().numpy())  # Tensor에서 numpy로 변환
            passenger_ids.extend(passenger_ids_batch.cpu().numpy())  # Tensor에서 numpy로 변환

    # 예측 결과를 DataFrame으로 변환
    preds_df = pd.DataFrame({
        "PassengerId": passenger_ids,
        "Survived": predictions
    })
    
    # DataFrame을 CSV 파일로 저장
    preds_df.to_csv(output_path, index=False)
    print(f"Test predictions saved to {os.path.abspath(output_path)}")  # 파일의 절대 경로 출력

    return predictions


In [73]:
# version 1
def main(args):
    wandb.init(project="titanic_model_training", config={
        "epochs": args.epochs,
        "batch_size": args.batch_size,
        "learning_rate": args.learning_rate
    })

    # 전처리된 데이터셋 로드
    all_df = get_preprocessed_dataset()
    
    # 훈련, 검증, 테스트 데이터셋 분리
    train_df = all_df[all_df['Survived'].notna()]  # Survived 값이 있는 행을 훈련용 데이터로 사용
    test_df = all_df[all_df['Survived'].isna()]    # Survived 값이 없는 행을 테스트 데이터로 사용
    
    # 특징(X)와 타겟(y) 분리
    X_train = train_df.drop(columns=['Survived']).values
    y_train = train_df['Survived'].values

    X_test = test_df.drop(columns=['Survived']).values
    # 테스트 데이터는 타겟(y) 값이 없으므로 y_test를 설정하지 않음

    # Dataset 생성
    full_train_dataset = TitanicDataset(X_train, y_train)
    
    # 훈련/검증 데이터 분리 (예: 80% 훈련, 20% 검증)
    train_size = int(0.8 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size
    train_dataset, validation_dataset = random_split(full_train_dataset, [train_size, val_size])
    
    test_dataset = TitanicTestDataset(X_test)

    # DataLoader 생성
    train_data_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    validation_data_loader = DataLoader(validation_dataset, batch_size=args.batch_size, shuffle=True)
    test_data_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

    # 모델 및 옵티마이저 초기화
    model = MyModel(n_input=X_train.shape[1], n_output=2)  # 출력 크기를 2로 설정해 이진 분류 수행
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

    # 훈련 루프 실행
    training_loop(model, optimizer, train_data_loader, validation_data_loader)

    # 테스트 데이터에 대한 예측 수행 및 결과 출력
    test_predictions = predict_test(model, test_data_loader)
    print(f"Test predictions: {test_predictions}")

    wandb.finish()


In [84]:
# version 2 main함수
def main(args):
    wandb.init(project="titanic_model_training", config={
        "epochs": args.epochs,
        "batch_size": args.batch_size,
        "learning_rate": args.learning_rate
    })

    # 전처리된 데이터셋 로드
    all_df = get_preprocessed_dataset()
    
    # 훈련, 검증, 테스트 데이터셋 분리
    train_df = all_df[all_df['Survived'].notna()]  # Survived 값이 있는 행을 훈련용 데이터로 사용
    test_df = all_df[all_df['Survived'].isna()]    # Survived 값이 없는 행을 테스트 데이터로 사용
    
    # 특징(X)와 타겟(y) 분리
    X_train = train_df.drop(columns=['Survived']).values
    y_train = train_df['Survived'].values

    X_test = test_df.drop(columns=['Survived']).values
    passenger_ids_test = test_df['PassengerId'].values  # 테스트 데이터에서 PassengerId 가져오기

    # Dataset 생성
    full_train_dataset = TitanicDataset(X_train, y_train)
    
    # 훈련/검증 데이터 분리 (예: 80% 훈련, 20% 검증)
    train_size = int(0.8 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size
    train_dataset, validation_dataset = random_split(full_train_dataset, [train_size, val_size])
    
    # 테스트 데이터셋에 PassengerId 포함
    test_dataset = TitanicTestDataset(X_test, passenger_ids_test)

    # DataLoader 생성
    train_data_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    validation_data_loader = DataLoader(validation_dataset, batch_size=args.batch_size, shuffle=True)
    test_data_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

    # 모델 및 옵티마이저 초기화
    model = MyModel(n_input=X_train.shape[1], n_output=2)  # 출력 크기를 2로 설정해 이진 분류 수행
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

    # 훈련 루프 실행
    training_loop(model, optimizer, train_data_loader, validation_data_loader)

    # 테스트 데이터에 대한 예측 수행 및 결과 출력
    test_predictions = predict_test(model, test_data_loader)
    print(f"Test predictions: {test_predictions}")

    wandb.finish()


In [110]:
# version 3
def main(args):
    wandb.init(project="titanic_model_training", config={
        "epochs": args.epochs,
        "batch_size": args.batch_size,
        "learning_rate": args.learning_rate
    })

    # 전처리된 데이터셋 로드
    all_df = get_preprocessed_dataset()
    
    # 훈련, 검증, 테스트 데이터셋 분리
    train_df = all_df[all_df['Survived'].notna()]  # Survived 값이 있는 행을 훈련용 데이터로 사용
    test_df = all_df[all_df['Survived'].isna()]    # Survived 값이 없는 행을 테스트 데이터로 사용
    
    # 특징(X)와 타겟(y) 분리
    X_train = train_df.drop(columns=['Survived']).values
    y_train = train_df['Survived'].values

    X_test = test_df.drop(columns=['Survived']).values
    passenger_ids_test = test_df['PassengerId'].values  # 테스트 데이터에서 PassengerId 가져오기

    # Dataset 생성
    full_train_dataset = TitanicDataset(X_train, y_train)
    
    # 훈련/검증 데이터 분리 (예: 80% 훈련, 20% 검증)
    train_size = int(0.8 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size
    train_dataset, validation_dataset = random_split(full_train_dataset, [train_size, val_size])
    
    # 테스트 데이터셋에 PassengerId 포함
    test_dataset = TitanicTestDataset(X_test, passenger_ids_test)

    # DataLoader 생성
    train_data_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    validation_data_loader = DataLoader(validation_dataset, batch_size=args.batch_size, shuffle=True)
    test_data_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

    # 모델 및 옵티마이저 초기화
    model = MyModel(n_input=X_train.shape[1], n_output=2)  # 출력 크기를 2로 설정해 이진 분류 수행
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

    # 훈련 루프 실행
    training_loop(model, optimizer, train_data_loader, validation_data_loader)

    # 테스트 데이터에 대한 예측 수행 및 결과 출력
    test_predictions = predict_test(model, test_data_loader)
    print(f"Test predictions: {test_predictions}")

    wandb.finish()


In [113]:
# Section 7: Argument Parsing and Main Execution
if __name__ == "__main__":
    # argparse.Namespace로 인자를 직접 설정합니다.
    args = argparse.Namespace(
        train_csv_path="train.csv",                # 실제 훈련 데이터 경로
        test_csv_path="test.csv",                  # 실제 테스트 데이터 경로
        output_path="test_predictions_ELU_testing_almost_final.csv",    # 테스트 예측 결과 파일 경로
        wandb=True,                                # WandB 사용 여부 (True로 설정하면 WandB 사용)
        batch_size=64,                             # 배치 사이즈
        epochs=300,                                # 학습 에폭 수
        learning_rate=0.001,                       # 학습률
        hidden_units=[128, 64]                     # 은닉층 크기 리스트
    )

    # main 함수 호출
    main(args)


Epoch 1 - Training Loss: 0.7086, Validation Loss: 0.7042
Epoch 2 - Training Loss: 0.6937, Validation Loss: 0.7016
Epoch 3 - Training Loss: 0.7023, Validation Loss: 0.7094
Epoch 4 - Training Loss: 0.6840, Validation Loss: 0.7077
Epoch 5 - Training Loss: 0.6916, Validation Loss: 0.7054
Epoch 6 - Training Loss: 0.6914, Validation Loss: 0.7046
Epoch 7 - Training Loss: 0.7078, Validation Loss: 0.6990
Epoch 8 - Training Loss: 0.6900, Validation Loss: 0.6987
Epoch 9 - Training Loss: 0.6907, Validation Loss: 0.6924
Epoch 10 - Training Loss: 0.6997, Validation Loss: 0.7028
Epoch 11 - Training Loss: 0.6912, Validation Loss: 0.6954
Epoch 12 - Training Loss: 0.6709, Validation Loss: 0.6956
Epoch 13 - Training Loss: 0.7892, Validation Loss: 0.9253
Epoch 14 - Training Loss: 0.8123, Validation Loss: 0.7048
Epoch 15 - Training Loss: 0.7005, Validation Loss: 0.7057
Epoch 16 - Training Loss: 0.6842, Validation Loss: 0.6991
Epoch 17 - Training Loss: 0.7018, Validation Loss: 0.7004
Epoch 18 - Training Los

Epoch,▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██████
Training Loss,▂▂▄▅▂▂▁▁▂▂▁▁▂▂▂▁▂▁▁▂▂▁▂▁▇███▂▁▁▂▁▂▁▂▂▁▁▁
Validation Loss,▂▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁█████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,300
Training Loss,0.68729
Validation Loss,0.68873


# Section 7: Argument Parsing and Main Execution

In [114]:
# Section 7: Argument Parsing and Main Execution
if __name__ == "__main__":
    # argparse.Namespace로 인자를 직접 설정합니다.
    args = argparse.Namespace(
        train_csv_path="train.csv",                # 실제 훈련 데이터 경로
        test_csv_path="test.csv",                  # 실제 테스트 데이터 경로
        output_path="test_predictions_ELU_testing_almost_final.csv",    # 테스트 예측 결과 파일 경로
        wandb=True,                                # WandB 사용 여부 (True로 설정하면 WandB 사용)
        batch_size=32,                             # 배치 사이즈
        epochs=250,                                # 학습 에폭 수
        learning_rate=0.0005,                       # 학습률
        hidden_units=[128, 64]                     # 은닉층 크기 리스트
    )

    # main 함수 호출
    main(args)


Epoch 1 - Training Loss: 0.7273, Validation Loss: 0.6761
Epoch 2 - Training Loss: 0.7153, Validation Loss: 0.6725
Epoch 3 - Training Loss: 0.6941, Validation Loss: 0.6906
Epoch 4 - Training Loss: 0.7089, Validation Loss: 0.6802
Epoch 5 - Training Loss: 0.7034, Validation Loss: 0.6774
Epoch 6 - Training Loss: 0.7057, Validation Loss: 0.6814
Epoch 7 - Training Loss: 0.6917, Validation Loss: 0.6690
Epoch 8 - Training Loss: 0.6961, Validation Loss: 0.6610
Epoch 9 - Training Loss: 0.6941, Validation Loss: 0.6803
Epoch 10 - Training Loss: 0.6963, Validation Loss: 0.6726
Epoch 11 - Training Loss: 0.7009, Validation Loss: 0.6564
Epoch 12 - Training Loss: 0.6911, Validation Loss: 0.6676
Epoch 13 - Training Loss: 0.6904, Validation Loss: 0.6686
Epoch 14 - Training Loss: 0.6930, Validation Loss: 0.6695
Epoch 15 - Training Loss: 0.6990, Validation Loss: 0.6764
Epoch 16 - Training Loss: 0.7081, Validation Loss: 0.6731
Epoch 17 - Training Loss: 0.6969, Validation Loss: 0.6705
Epoch 18 - Training Los

Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▇▇▇▇██████
Training Loss,█▅▅▄▄▃▄▄▅▃▃▅▄▄▄▃▄▃▃▄▃▃▂▂▃▆▃▃▃▄▂▁▃▃▂▃▃▃▂▃
Validation Loss,█▄▁▆▅▆▅▄▂▆▃▄▅▅▆▅▄▅▅▃▃▃▃▄▃▁▃▃▂▃▃▂▂▃▄▃▄▂▄▃
Epoch,250
Training Loss,0.67918
Validation Loss,0.67115


![competition78](./78Scores_leaderboard2.png)

# 새로 시작

# Section 1: Imports and Dataset Class Definitions

In [126]:
# Section 1: Imports and Dataset Class Definitions
import os
import torch
import pandas as pd
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from datetime import datetime
import wandb
import argparse
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Section 2: # TitanicDataset Class for Train and Validation Data

In [127]:
# TitanicDataset Class for Train and Validation Data
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return {'input': self.X[idx], 'target': self.y[idx]}
    
    def __str__(self):
        return f"Data Size: {len(self.X)}, Input Shape: {self.X.shape}, Target Shape: {self.y.shape}"

# Section 3: TitanicTestDataset

In [128]:
class TitanicTestDataset(Dataset):
    def __init__(self, X, passenger_ids):
        self.X = torch.FloatTensor(X)
        self.passenger_ids = passenger_ids  # PassengerId 추가
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        feature = self.X[idx]
        passenger_id = self.passenger_ids[idx]  # PassengerId 반환
        return {'input': feature, 'PassengerId': passenger_id}  # PassengerId와 입력 특징 반환

# Section 4: get_preprocessed_dataset_1-6

In [129]:
def get_preprocessed_dataset_1(df):
    # 성별 인코딩 (male=0, female=1)
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    return df

def get_preprocessed_dataset_2(df):
    # 결측값 처리 (Age의 결측값을 평균으로 채움)
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    return df

def get_preprocessed_dataset_3(df):
    # 결측값 처리 (Fare의 결측값을 평균으로 채움)
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    return df

def get_preprocessed_dataset_4(df):
    # 불필요한 열 제거, 단 PassengerId는 제거하지 않음
    columns_to_drop = ['Name', 'Ticket', 'Cabin', 'Embarked']  # 'PassengerId' 제외
    df = df.drop(columns=columns_to_drop, axis=1, errors='ignore')
    return df

def get_preprocessed_dataset_5(df):
    # Pclass를 범주형에서 연속형으로 처리
    df['Pclass'] = df['Pclass'].astype(float)
    return df

def get_preprocessed_dataset_6(df):
    # 정규화 (필요한 경우)
    scaler = StandardScaler()
    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
    df[features] = scaler.fit_transform(df[features])
    return df


# Section 5 : get_preprocessed_dataset

In [130]:
# Section 2: Data Preprocessing
def get_preprocessed_dataset():
    CURRENT_FILE_PATH = os.getcwd()
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
    
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    all_df = pd.concat([train_df, test_df], sort=False)
    
    # Call pre-processing functions (define as needed)
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)
    
    return all_df

# Section 6 : Model Definition

In [131]:
# Section 3: Model Definition version 2
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(n_input, 256),
            nn.ELU(),
            nn.Dropout(0.3),  # Dropout 추가
            nn.Linear(256, 64),
            nn.ELU(),
            nn.Dropout(0.3),  # Dropout 추가
            nn.Linear(64, n_output),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.model(x)

In [132]:
# Section 4: Training and Evaluation Loops_v2_wandB에 추가하기
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(1, n_epochs + 1):
        model.train()
        train_loss = 0.0
        for batch in train_data_loader:
            inputs, targets = batch['input'], batch['target']
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            train_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_data_loader)

        # Validation Loss 계산
        model.eval()
        validation_loss = 0.0
        with torch.no_grad():
            for batch in validation_data_loader:
                inputs, targets = batch['input'], batch['target']
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                validation_loss += loss.item()
        
        validation_loss /= len(validation_data_loader)

        # WandB에 기록
        wandb.log({"Training Loss": train_loss, "Validation Loss": validation_loss, "Epoch": epoch})

        print(f"Epoch {epoch} - Training Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}")


# Section 7 : predict_test

In [133]:
# predict_test version 5
# Section 5: Model Testing and Predictio
import pandas as pd

# Model Testing and Prediction
def predict_test(model, test_data_loader, output_path="test_predictions_ELU_testing_almost_final.csv"):
    model.eval()
    predictions = []
    passenger_ids = []  # PassengerId 저장을 위한 리스트

    # 예측 수행
    with torch.no_grad():
        for batch in test_data_loader:
            inputs = batch['input']
            passenger_ids_batch = batch['PassengerId']  # PassengerId 가져오기
            outputs = model(inputs)
            predicted_labels = torch.argmax(outputs, dim=1)
            
            # 예측 값과 PassengerId 저장
            predictions.extend(predicted_labels.cpu().numpy())  # Tensor에서 numpy로 변환
            passenger_ids.extend(passenger_ids_batch.cpu().numpy())  # Tensor에서 numpy로 변환

    # 예측 결과를 DataFrame으로 변환
    preds_df = pd.DataFrame({
        "PassengerId": passenger_ids,
        "Survived": predictions
    })
    
    # DataFrame을 CSV 파일로 저장
    preds_df.to_csv(output_path, index=False)
    print(f"Test predictions saved to {os.path.abspath(output_path)}")  # 파일의 절대 경로 출력

    return predictions


# Section 8 : main function

In [134]:
# version 3
def main(args):
    wandb.init(project="titanic_model_training", config={
        "epochs": args.epochs,
        "batch_size": args.batch_size,
        "learning_rate": args.learning_rate
    })

    # 전처리된 데이터셋 로드
    all_df = get_preprocessed_dataset()
    
    # 훈련, 검증, 테스트 데이터셋 분리
    train_df = all_df[all_df['Survived'].notna()]  # Survived 값이 있는 행을 훈련용 데이터로 사용
    test_df = all_df[all_df['Survived'].isna()]    # Survived 값이 없는 행을 테스트 데이터로 사용
    
    # 특징(X)와 타겟(y) 분리
    X_train = train_df.drop(columns=['Survived']).values
    y_train = train_df['Survived'].values

    X_test = test_df.drop(columns=['Survived']).values
    passenger_ids_test = test_df['PassengerId'].values  # 테스트 데이터에서 PassengerId 가져오기

    # Dataset 생성
    full_train_dataset = TitanicDataset(X_train, y_train)
    
    # 훈련/검증 데이터 분리 (예: 80% 훈련, 20% 검증)
    train_size = int(0.8 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size
    train_dataset, validation_dataset = random_split(full_train_dataset, [train_size, val_size])
    
    # 테스트 데이터셋에 PassengerId 포함
    test_dataset = TitanicTestDataset(X_test, passenger_ids_test)

    # DataLoader 생성
    train_data_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    validation_data_loader = DataLoader(validation_dataset, batch_size=args.batch_size, shuffle=True)
    test_data_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

    # 모델 및 옵티마이저 초기화
    model = MyModel(n_input=X_train.shape[1], n_output=2)  # 출력 크기를 2로 설정해 이진 분류 수행
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

    # 훈련 루프 실행
    training_loop(model, optimizer, train_data_loader, validation_data_loader)

    # 테스트 데이터에 대한 예측 수행 및 결과 출력
    test_predictions = predict_test(model, test_data_loader)
    print(f"Test predictions: {test_predictions}")

    wandb.finish()


# Section 9: Argument Parsing and Main Execution

In [ ]:
if __name__ == "__main__":
    # argparse.Namespace로 인자를 직접 설정합니다.
    args = argparse.Namespace(
        train_csv_path="train.csv",                # 실제 훈련 데이터 경로
        test_csv_path="test.csv",                  # 실제 테스트 데이터 경로
        output_path="test_predictions_ELU_testing_almost_final.csv",    # 테스트 예측 결과 파일 경로
        wandb=True,                                # WandB 사용 여부 (True로 설정하면 WandB 사용)
        batch_size=64,                             # 배치 사이즈
        epochs=300,                                # 학습 에폭 수
        learning_rate=0.001,                       # 학습률
        hidden_units=[128, 64]                     # 은닉층 크기 리스트
    )

    # main 함수 호출
    main(args)

In [135]:
# Section 7: Argument Parsing and Main Execution
if __name__ == "__main__":
    # argparse.Namespace로 인자를 직접 설정합니다.
    args = argparse.Namespace(
        train_csv_path="train.csv",                # 실제 훈련 데이터 경로
        test_csv_path="test.csv",                  # 실제 테스트 데이터 경로
        output_path="test_predictions_ELU_testing_almost_final.csv",    # 테스트 예측 결과 파일 경로
        wandb=True,                                # WandB 사용 여부 (True로 설정하면 WandB 사용)
        batch_size=32,                             # 배치 사이즈
        epochs=250,                                # 학습 에폭 수
        learning_rate=0.0005,                       # 학습률
        hidden_units=[128, 64]                     # 은닉층 크기 리스트
    )

    # main 함수 호출
    main(args)


Epoch 1 - Training Loss: 0.6978, Validation Loss: 0.6836
Epoch 2 - Training Loss: 0.6960, Validation Loss: 0.7085
Epoch 3 - Training Loss: 0.7061, Validation Loss: 0.6979
Epoch 4 - Training Loss: 0.6939, Validation Loss: 0.6992
Epoch 5 - Training Loss: 0.6979, Validation Loss: 0.6882
Epoch 6 - Training Loss: 0.6931, Validation Loss: 0.6866
Epoch 7 - Training Loss: 0.6958, Validation Loss: 0.6913
Epoch 8 - Training Loss: 0.6967, Validation Loss: 0.6953
Epoch 9 - Training Loss: 0.6991, Validation Loss: 0.6897
Epoch 10 - Training Loss: 0.6909, Validation Loss: 0.6949
Epoch 11 - Training Loss: 0.6798, Validation Loss: 0.6794
Epoch 12 - Training Loss: 0.6916, Validation Loss: 0.6879
Epoch 13 - Training Loss: 0.6968, Validation Loss: 0.6849
Epoch 14 - Training Loss: 0.6976, Validation Loss: 0.6862
Epoch 15 - Training Loss: 0.7814, Validation Loss: 0.6785
Epoch 16 - Training Loss: 0.7337, Validation Loss: 0.6963
Epoch 17 - Training Loss: 0.6850, Validation Loss: 0.6927
Epoch 18 - Training Los

Epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
Training Loss,▃▃▃█▂▃▃▂▃▂▂▃▂▂▂▃▂▁▂▂▁▂▂▂▂▂▂▁▂▂▂▂▃▂▁▂▂▂▁▁
Validation Loss,▄▅▆▅▇▆█▇▆▆▅▅▅▃▄▄▆▅▅▃▆▆▅▄▅▄▂▅▅▁▄▁▅▇▄▃▄▄▁▄
Epoch,250
Training Loss,0.67535
Validation Loss,0.66484


## 발견

- Leaderboard의 경우 마지막 제출을 기준으로 판단하기도 함. 

![competition78](./78Scores_leaderboard2.png)

## 에러 발생

발견 - 하루 10개 이상 제출을 못함 

![competition78](./Per_day_allow.png)

# 소감

많은 시간을 투자하진 못했지만 그래도 짧은 시간에 최대한 시도해보려고 했던게 epoch를 200~400정도까지 바꿔가며 추세를 판단했습니다.

batch 수는 한번만 하긴했지만 예를 들어 32라든가, learning rate도 0.0005로도 했던 것 같은데 시간이 오래걸리는만큼 세밀하게 학습할거라는 기대를 했지만 왜인지는 몰라도 그렇게 분석 정확도가 높지가 않던데 어떤식으로 해야 올라갈까를 고민하게된 계기가 되었으며 여러가지로 어떻게하면 올릴 수 있을까와. 어떤것을 올리면 어떤것도 영향을 끼칠까도 고민을 하고 시도하고 싶은 계기가 되었습니다. 

여러가지로 본인이 아는게 정말 ㅇ
감사합니다. 